# 处理中文，并使用jieba分词

In [7]:
from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import jieba

In [8]:
sentences = [
    "今天的天气晴朗，天空湛蓝。",
    "明天的预报预测将有大雨和雷暴。",
    "我喜欢在温和的天气下观看日落。",
    "自然语言处理是计算机科学的一个研究领域。",
    "NLP技术被广泛用于文本分析和语言理解。",
    "天气预报依赖于复杂的算法和数据分析。",
    "了解天气模式对农业和灾害管理至关重要。",
    "NLP算法可以帮助分析社交媒体帖子的情感。",
    "天气可能会变化无常，尤其是在季节交替时。",
    "像BERT和GPT-3这样的NLP模型已经彻底改变了语言理解任务。",
    "极端天气事件如飓风和龙卷风需要先进的预测模型。",
    "NLP被用于像Siri和Alexa这样的虚拟助手，以理解人类命令。",
    "气候变化正在影响全球的天气模式。",
    "NLP可以帮助进行机器翻译，使跨语言交流变得更容易。",
    "气象卫星提供实时数据供气象学家分析。",
    "语义分析是自然语言处理的重要组成部分。",
    "天气现象如厄尔尼诺影响着全球的天气。",
    "词性标注是NLP中的一个基本任务。",
    "沿海地区的天气常受海洋洋流影响。",
    "NLP有助于聊天机器人生成类似人类的回复。"
]

https://maartengr.github.io/BERTopic/api/bertopic.html
The default embedding model is all-MiniLM-L6-v2 when selecting language="english" and paraphrase-multilingual-MiniLM-L12-v2 when selecting language="multilingual".

# 使用能处理中文的词嵌入模型

In [9]:
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
hdbscan_model = HDBSCAN(min_cluster_size=2)

# 使用jieba分词
https://maartengr.github.io/BERTopic/faq.html#how-can-i-use-bertopic-with-chinese-documents
How can I use BERTopic with Chinese documents?
Currently, CountVectorizer tokenizes text by splitting whitespace which does not work for Chinese. To get it to work, you will have to create a custom CountVectorizer with jieba:


from sklearn.feature_extraction.text import CountVectorizer
import jieba

def tokenize_zh(text):
    words = jieba.lcut(text)
    return words

vectorizer = CountVectorizer(tokenizer=tokenize_zh)

In [10]:
def tokenize_zh(text):
    words = jieba.lcut(text) # 切词，并返回list
    print('切词：', words)
    return words

vectorizer_model = CountVectorizer(tokenizer=tokenize_zh)

In [11]:
# 这里有两个问题
# 其一是每次运行模型都会执行切词操作，非常浪费时间
# 其二是介词会包含：标点符号、的、了等虚词，这些词更应该被排除在外
topic_model = BERTopic(
  embedding_model=embedding_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
)

topic_model.fit_transform(sentences) # 训练模型
topic_model.get_topic_info() # 获取主题聚类信息

切词： ['今天', '的', '天气晴朗', '，', '天空', '湛蓝', '。', ' ', '明天', '的', '预报', '预测', '将', '有', '大雨', '和', '雷暴', '。', ' ', '我', '喜欢', '在', '温和', '的', '天气', '下', '观看', '日落', '。', ' ', '天气预报', '依赖于', '复杂', '的', '算法', '和', '数据分析', '。', ' ', '了解', '天气', '模式', '对', '农业', '和', '灾害', '管理', '至关重要', '。', ' ', '天气', '可能', '会', '变化无常', '，', '尤其', '是', '在', '季节', '交替', '时', '。', ' ', '极端', '天气', '事件', '如', '飓风', '和', '龙卷风', '需要', '先进', '的', '预测', '模型', '。', ' ', '气候变化', '正在', '影响', '全球', '的', '天气', '模式', '。', ' ', '气象卫星', '提供', '实时', '数据', '供', '气象学家', '分析', '。', ' ', '天气现象', '如', '厄尔尼诺', '影响', '着', '全球', '的', '天气', '。', ' ', '沿海地区', '的', '天气', '常受', '海洋', '洋流', '影响', '。']
切词： ['自然语言', '处理', '是', '计算机科学', '的', '一个', '研究', '领域', '。', ' ', 'nlp', '技术', '被', '广泛', '用于', '文本', '分析', '和', '语言', '理解', '。', ' ', 'nlp', '算法', '可以', '帮助', '分析', '社交', '媒体', '帖子', '的', '情感', '。', ' ', '像', 'bert', '和', 'gpt', '-', '3', '这样', '的', 'nlp', '模型', '已经', '彻底改变', '了', '语言', '理解', '任务', '。', ' ', 'nlp', '被', '用于', '像', 'siri', 

,Topic,Count,Name,Representation,Representative_Docs
0,0,11,0_。_天气_ _的,"[。, 天气, , 的, 和, 影响, 在, 全球, 如, 模式]","[极端天气事件如飓风和龙卷风需要先进的预测模型。, 气候变化正在影响全球的天气模式。, 天气..."
1,1,9,1_nlp_。_ _的,"[nlp, 。, , 的, 语言, 理解, 分析, 是, 和, 处理]","[NLP技术被广泛用于文本分析和语言理解。, NLP被用于像Siri和Alexa这样的虚拟助..."


In [12]:
topic_docs = topic_model.get_document_info(sentences)
topic_docs.to_csv('./聚类结果.csv')